In [1]:
import os
os.chdir('..')

In [2]:
import collections
import functools 

import tensorflow_federated as tff
import tensorflow as tf

from FederatedEMNISTLearner import FederatedEMNISTLearnerConfig

In [3]:
(emnist_train, emnist_test,) = tff.simulation.datasets.emnist.load_data()

/opt/conda/lib/python3.7/site-packages/h5py/_hl/dataset.py:313: H5pyDeprecationWarning: dataset.value has been deprecated. Use dataset[()] instead.
  "Use dataset[()] instead.", H5pyDeprecationWarning)


In [4]:
config = FederatedEMNISTLearnerConfig(N_CLIENTS=7)

In [5]:
%time
def preprocess(dataset):
    def element_fn(element):
        return collections.OrderedDict(
            [
                ("x", tf.reshape(element["pixels"], [-1])),
                ("y", tf.reshape(element["label"], [1])),
            ]
        )

    return (
        dataset.repeat(config.N_ROUNDS)
        .map(element_fn)
        .shuffle(config.SHUFFLE_BUFFER)
        .batch(config.BATCH_SIZE)
    )


def get_federated_data(dataset):
    federated_data = [list() for _ in range(config.N_CLIENTS)]
    for i, client_id in enumerate(dataset.client_ids):
        client_data = preprocess(dataset.create_tf_dataset_for_client(client_id))
        federated_data[i % config.N_CLIENTS].append(client_data)
    return [functools.reduce(lambda a, b: a.concatenate(b), fd) for fd in federated_data]

# def concatenate_datasets(datasets):
#     dataset = datasets[0]
#     if len(datasets) == 1:
#         return dataset
#     for d in datasets[1:]:
#         dataset.concatenate(d)


# federated_train_data = get_federated_data(emnist_train)
# federated_test_data = get_federated_data(emnist_test)

CPU times: user 6 µs, sys: 0 ns, total: 6 µs
Wall time: 11.2 µs


In [10]:
dataset = emnist_train
federated_data = [list() for _ in range(config.N_CLIENTS)]
for i, client_id in enumerate(dataset.client_ids):
    client_data = preprocess(dataset.create_tf_dataset_for_client(client_id))
    federated_data[i % config.N_CLIENTS].append(client_data)
    if i == 30:
        break
res = [functools.reduce(lambda a, b: a.concatenate(b), fd) for fd in federated_data]

In [12]:
ds = functools.reduce(lambda a, b: a.concatenate(b), federated_data[0])

In [18]:
ds.take(2)

<TakeDataset shapes: OrderedDict([(x, (None, 784)), (y, (None, 1))]), types: OrderedDict([(x, tf.float32), (y, tf.int32)])>

In [8]:
import pickle

with open('federated_train_data', 'wb') as f:
    pickle.dump(res, f)

InternalError: Tensorflow type 21 not convertible to numpy dtype.

In [19]:
import sys
sys.getrecursionlimit()

3000

In [1]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))

Num GPUs Available:  1
